### Unconstrained minimization of multivariate scalar functions (minimize)

### Import library

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

### Define functions we will use

In [2]:
def rosen(x):
    """The Rosenbrock function"""
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

def rosen_der(x):
    """The Rosenbrock Derivative function"""
    xm = x[1:-1]
    xm_m1 = x[:-2]
    xm_p1 = x[2:]
    der = np.zeros_like(x)
    der[1:-1] = 200*(xm-xm_m1**2) - 400*(xm_p1 - xm**2)*xm - 2*(1-xm)
    der[0] = -400*x[0]*(x[1]-x[0]**2) - 2*(1-x[0])
    der[-1] = 200*(x[-1]-x[-2]**2)
    return der

def rosen_hess(x):
    """The Rosenbrock Hessian function"""
    x = np.asarray(x)
    H = np.diag(-400*x[:-1],1) - np.diag(400*x[:-1],-1)
    diagonal = np.zeros_like(x)
    diagonal[0] = 1200*x[0]**2-400*x[1]+2
    diagonal[-1] = 200
    diagonal[1:-1] = 202 + 1200*x[1:-1]**2 - 400*x[2:]
    H = H + np.diag(diagonal)
    return H

### Nelder-Mead Simplex algorithm (method='Nelder-Mead')

The Nelder–Mead method (also downhill simplex method, amoeba method, or polytope method) is a commonly applied numerical method used to find the minimum or maximum of an objective function in a multidimensional space.

The Nelder-Mead is an algorithm for optimizing nonlinear which was published 1 by John Nelder and Roger Mead  (in) in 1965 . It is a heuristic numerical method which seeks to minimize a continuous function in a space with several dimensions.

Also called downhill simplex method , the algorithm exploits the concept of simplex which is a polytope of N +1 vertices in an N- dimensional space . Starting initially from such a simplex, it undergoes simple transformations during the iterations  : it deforms, moves and gradually reduces until its vertices approach a point where the function is locally minimal.

In [3]:
x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])
res = minimize(rosen, x0, method='nelder-mead',
               options={'xatol': 1e-8, 'disp': True})
print(res.x)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 339
         Function evaluations: 571
[1. 1. 1. 1. 1.]


### Broyden-Fletcher-Goldfarb-Shanno algorithm (method='BFGS')

In mathematics , the Broyden-Fletcher-Goldfarb-Shanno ( BFGS ) method is a method for solving a nonlinear optimization problem without constraints.

The BFGS method is a solution often used when one wants an algorithm with directions of descent .

The main idea of ​​this method is to avoid explicitly building the Hessian matrix and instead to build an approximation of the inverse of the second derivative of the function to be minimized, by analyzing the different successive gradients . This approximation of the derivatives of the function leads to a quasi-Newton method (a variant of Newton's method ) so as to find the minimum in the parameter space.

The Hessian matrix does not need to be recomputed at each iteration of the algorithm. However, the method assumes that the function can be locally approximated by a quadratic limited expansion around the optimum.

In order to converge more quickly to the solution, this routine uses the gradient of the objective function. If the gradient is not given by the user, then it is estimated using first-differences. The Broyden-Fletcher-Goldfarb-Shanno (BFGS) method typically requires fewer function calls than the simplex algorithm even when the gradient must be estimated.

In [4]:
res = minimize(rosen, x0, method='BFGS', jac=rosen_der,
               options={'disp': True})
print(res.x)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 25
         Function evaluations: 30
         Gradient evaluations: 30
[1.00000004 1.0000001  1.00000021 1.00000044 1.00000092]


### Newton-Conjugate-Gradient algorithm (method='Newton-CG')

In numerical analysis , the conjugate gradient method is an algorithm for solving systems of linear equations whose matrix is symmetrically positive definite . This method, imagined simultaneously in 1950 by Cornelius Lanczos , Eduard Stiefel and Magnus Hestenes 1 , is an iterative method which converges in a finite number of iterations (at most equal to the dimension of the linear system). However, its great practical interest from the point of view of computation time comes from the fact that a clever initialization (known as “preconditioning”) makes it possible to lead in only a few passages to an estimate very close to the exact solution, this is why in practice one limits oneself to a number of iterations much lower than the number of unknowns.

The biconjugate gradient method provides a generalization for unsymmetrical matrices.

Newton-Conjugate Gradient algorithm is a modified Newton’s method and uses a conjugate gradient algorithm to (approximately) invert the local Hessian [NW]. Newton’s method is based on fitting the function locally to a quadratic form.

In [5]:
res = minimize(rosen, x0, method='Newton-CG',
               jac=rosen_der, hess=rosen_hess,
               options={'xtol': 1e-8, 'disp': True})
print(res.x)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 24
         Function evaluations: 33
         Gradient evaluations: 56
         Hessian evaluations: 24
[1.         1.         1.         0.99999999 0.99999999]


In [6]:
def rosen_hess_p(x, p):
    x = np.asarray(x)
    Hp = np.zeros_like(x)
    Hp[0] = (1200*x[0]**2 - 400*x[1] + 2)*p[0] - 400*x[0]*p[1]
    Hp[1:-1] = -400*x[:-2]*p[:-2]+(202+1200*x[1:-1]**2-400*x[2:])*p[1:-1] \
               -400*x[1:-1]*p[2:]
    Hp[-1] = -400*x[-2]*p[-2] + 200*p[-1]
    return Hp

res = minimize(rosen, x0, method='Newton-CG',
               jac=rosen_der, hessp=rosen_hess_p,
               options={'xtol': 1e-8, 'disp': True})
print(res.x)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 24
         Function evaluations: 33
         Gradient evaluations: 56
         Hessian evaluations: 66
[1.         1.         1.         0.99999999 0.99999999]


### Trust-Region Newton-Conjugate-Gradient Algorithm (method='trust-ncg')

The Newton-CG method is a line search method: it finds a direction of search minimizing a quadratic approximation of the function and then uses a line search algorithm to find the (nearly) optimal step size in that direction. An alternative approach is to, first, fix the step size limit (Hessian) and then find the optimal step p inside the given trust-radius by solving the following quadratic subproblem

In [7]:
res = minimize(rosen, x0, method='trust-ncg',
               jac=rosen_der, hess=rosen_hess,
               options={'gtol': 1e-8, 'disp': True})
print(res.x)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 20
         Function evaluations: 21
         Gradient evaluations: 20
         Hessian evaluations: 19
[1. 1. 1. 1. 1.]


In [8]:
res = minimize(rosen, x0, method='trust-ncg',
               jac=rosen_der, hessp=rosen_hess_p,
               options={'gtol': 1e-8, 'disp': True})
print(res.x)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 20
         Function evaluations: 21
         Gradient evaluations: 20
         Hessian evaluations: 0
[1. 1. 1. 1. 1.]


### Trust-Region Truncated Generalized Lanczos / Conjugate Gradient Algorithm (method='trust-krylov')

Similar to the trust-ncg method, the trust-krylov method is a method suitable for large-scale problems as it uses the hessian only as linear operator by means of matrix-vector products. It solves the quadratic subproblem more accurately than the trust-ncg method.

In [9]:
res = minimize(rosen, x0, method='trust-krylov',
               jac=rosen_der, hess=rosen_hess,
               options={'gtol': 1e-8, 'disp': True})
print(res.x)

 iter inewton type    objective     ?g??_M?¹      leftmost         ?             ?             ?             ?             ?       
     0     0  cg_i -6.273083e+02  4.029038e+02  0.000000e+00  0.000000e+00  2.246107e+03  4.021147e+03  2.486853e-04  3.217671e-02

 iter inewton type    objective     ?g??_M?¹      leftmost         ?             ?             ?             ?             ?       
     0     0  cg_i -9.528585e+01  1.478412e+02  0.000000e+00  0.000000e+00  6.001708e+02  1.890129e+03  5.290645e-04  6.067939e-02

 iter inewton type    objective     ?g??_M?¹      leftmost         ?             ?             ?             ?             ?       
     0     0  cg_i -8.662599e+00  5.824611e+01  0.000000e+00  0.000000e+00  1.285783e+02  9.542387e+02  1.047956e-03  2.052100e-01

 iter inewton type    objective     ?g??_M?¹      leftmost         ?             ?             ?             ?             ?       
     0     0  cg_i -1.798598e+00  3.481545e+01  0.000000e+00  0.000000e+00  

In [10]:
res = minimize(rosen, x0, method='trust-krylov',
               jac=rosen_der, hessp=rosen_hess_p,
               options={'gtol': 1e-8, 'disp': True})
print(res.x)

 iter inewton type    objective     ?g??_M?¹      leftmost         ?             ?             ?             ?             ?       
     0     0  cg_i -6.273083e+02  4.029038e+02  0.000000e+00  0.000000e+00  2.246107e+03  4.021147e+03  2.486853e-04  3.217671e-02

 iter inewton type    objective     ?g??_M?¹      leftmost         ?             ?             ?             ?             ?       
     0     0  cg_i -9.528585e+01  1.478412e+02  0.000000e+00  0.000000e+00  6.001708e+02  1.890129e+03  5.290645e-04  6.067939e-02

 iter inewton type    objective     ?g??_M?¹      leftmost         ?             ?             ?             ?             ?       
     0     0  cg_i -8.662599e+00  5.824611e+01  0.000000e+00  0.000000e+00  1.285783e+02  9.542387e+02  1.047956e-03  2.052100e-01

 iter inewton type    objective     ?g??_M?¹      leftmost         ?             ?             ?             ?             ?       
     0     0  cg_i -1.798598e+00  3.481545e+01  0.000000e+00  0.000000e+00  

### Trust-Region Nearly Exact Algorithm (method='trust-exact')

All methods Newton-CG, trust-ncg and trust-krylov are suitable for dealing with large-scale problems (problems with thousands of variables). That is because the conjugate gradient algorithm approximately solve the trust-region subproblem (or invert the Hessian) by iterations without the explicit Hessian factorization. Since only the product of the Hessian with an arbitrary vector is needed, the algorithm is specially suited for dealing with sparse Hessians, allowing low storage requirements and significant time savings for those sparse problems.

For medium-size problems, for which the storage and factorization cost of the Hessian are not critical, it is possible to obtain a solution within fewer iteration by solving the trust-region subproblems almost exactly. To achieve that, a certain nonlinear equations is solved iteratively for each quadratic subproblem [CGT]. This solution requires usually 3 or 4 Cholesky factorizations of the Hessian matrix. As the result, the method converges in fewer number of iterations and takes fewer evaluations of the objective function than the other implemented trust-region methods. The Hessian product option is not supported by this algorithm. An example using the Rosenbrock function follows

In [11]:
res = minimize(rosen, x0, method='trust-exact',
               jac=rosen_der, hess=rosen_hess,
               options={'gtol': 1e-8, 'disp': True})

print(res.x)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 13
         Function evaluations: 14
         Gradient evaluations: 13
         Hessian evaluations: 14
[1. 1. 1. 1. 1.]
